In [ ]:
import boto3
import sagemaker.pytorch

# Access by job_name
job_name = '<TRAINING_JOB_NAME>'
pytorch_estimator = sagemaker.pytorch.estimator.PyTorch.attach(job_name, sagemaker_session=sagemaker.session.Session(boto_session=boto3.Session(region_name='ap-northeast-2')))

pytorch_model = sagemaker.pytorch.model.PyTorchModel(
    entry_point='batch_inference.py',
    name='suite-pytorch-inference-batch',
    model_data=pytorch_estimator.model_data,
    role=pytorch_estimator.role,
    framework_version=pytorch_estimator.framework_version,
    py_version=pytorch_estimator.py_version,
)

predictor = pytorch_model.deploy(
    instance_type='ml.g4dn.xlarge',
    initial_instance_count=1,
)

In [ ]:
import json, boto3
import sagemaker.pytorch


def predict(endpoint_name, body, content_type):
    client = boto3.client('sagemaker-runtime')
    outputs = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType=content_type,
        Accept='application/json',
        Body=body,
    )   
    return json.load(outputs['Body'])


req = {
    'auth': {
        'team_name': '<YOUR TEAM NAME>',
        'access_key': '<YOUR ACCESS KEY>',
        'project_name': '<YOUR PROJECT NAME>',
    },
    'label_ids': [
        '<YOUR LABEL ID>'
    ],
}


# Method 1

# Access by endpoint_name
endpoint_name = '<ENDPOINT_NAME>'
predictor = sagemaker.pytorch.model.PyTorchPredictor(endpoint_name)

predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

response = predictor.predict(req)
print(response)


# Method 2
response = predict(endpoint_name, json.dumps(req), 'application/json')
print(response)